In [1]:
import MetaTrader5 as mt
import pandas as pd
import plotly.express as px
import matplotlib.pylab as plt
import numpy as np
import talib
from talipp.indicators import EMA, SMA, Stoch, DPO

from datetime import datetime
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, confusion_matrix, classification_report
from own_functions import *

from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import roll_time_series, make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute

c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
mt.initialize()
login= 51505377
password="7JhZp8Vj"
server="ICMarketsSC-Demo"
mt.login(login,password,server)

False

In [3]:
symbol="USDCAD"
timeframe=mt.TIMEFRAME_D1

#ohcl data
ohlc_data=pd.DataFrame(mt.copy_rates_range(symbol,timeframe,datetime(2010,1,1),datetime.now()))
df=pd.DataFrame(ohlc_data)[['time','open','high','low','close']]

#Indicators
df['WILLR_15']=talib.WILLR(df['high'],df['low'],df['open'],timeperiod=15) #Williams' %R
df['WILLR_23']=talib.WILLR(df['high'],df['low'],df['open'],timeperiod=23) #Williams' %R
df['WILLR_42']=talib.WILLR(df['high'],df['low'],df['open'],timeperiod=42) #Williams' %R
df['WILLR_145']=talib.WILLR(df['high'],df['low'],df['open'],timeperiod=145) #Williams' %R

#Buy & Sell Flags
df['b_flag'] = 0
df['s_flag'] = 0

df=df.dropna()
df=df.reset_index(drop=True)

#Label Data: label_data(DataFrame, StopLoss, TakeProfit, Period, Symbol, Plot)
StopLoss=1
TakeProfit=1.25
BreakEvenRatio=StopLoss/(StopLoss+TakeProfit)
label_data(df,[StopLoss],[TakeProfit],80,symbol,False)
#print('BreatEvenRatio:', BreakEvenRatio)

KeyError: "None of [Index(['time', 'open', 'high', 'low', 'close'], dtype='object')] are in the [columns]"

In [ ]:
target='b_flag'
#target='s_flag'

selected_signal='WILLR_15'

df_melted = df[['time',selected_signal]]
df_melted["Symbols"] = symbol

df_rolled = roll_time_series(df_melted, column_id="Symbols", column_sort="time",
                             max_timeshift=20, min_timeshift=5)

X = extract_features(df_rolled.drop("Symbols", axis=1), 
                     column_id="id", column_sort="time", column_value=selected_signal, 
                     impute_function=impute, show_warnings=False)

X = X.set_index(X.index.map(lambda x: x[1]), drop=True)
X.index.name = "time"
X=X.dropna()

""" selected_signal='WILLR_23'


df_melted = df[['time',selected_signal]]
df_melted["Symbols"] = symbol

df_rolled = roll_time_series(df_melted, column_id="Symbols", column_sort="time",
                             max_timeshift=20, min_timeshift=5)

X2 = extract_features(df_rolled.drop("Symbols", axis=1), 
                     column_id="id", column_sort="time", column_value=selected_signal, 
                     impute_function=impute, show_warnings=False)

X2 = X2.set_index(X2.index.map(lambda x: x[1]), drop=True)
X2.index.name = "time"
X2=X2.dropna() """

selected_signal='WILLR_42'


df_melted = df[['time',selected_signal]]
df_melted["Symbols"] = symbol

df_rolled = roll_time_series(df_melted, column_id="Symbols", column_sort="time",
                             max_timeshift=20, min_timeshift=5)

X3 = extract_features(df_rolled.drop("Symbols", axis=1), 
                     column_id="id", column_sort="time", column_value=selected_signal, 
                     impute_function=impute, show_warnings=False)

X3 = X3.set_index(X3.index.map(lambda x: x[1]), drop=True)
X3.index.name = "time"
X3=X3.dropna()

""" selected_signal='WILLR_145'


df_melted = df[['time',selected_signal]]
df_melted["Symbols"] = symbol

df_rolled = roll_time_series(df_melted, column_id="Symbols", column_sort="time",
                             max_timeshift=20, min_timeshift=5)

X4 = extract_features(df_rolled.drop("Symbols", axis=1), 
                     column_id="id", column_sort="time", column_value=selected_signal, 
                     impute_function=impute, show_warnings=False)

X4 = X4.set_index(X4.index.map(lambda x: x[1]), drop=True)
X4.index.name = "time"
X4=X4.dropna() """

df=df.set_index('time')
df = df[df.index.isin(X.index)]
X = select_features(X, df[target])
#X2 = select_features(X2, df[target])
X3 = select_features(X3, df[target])
#X4 = select_features(X4, df[target])
#df = pd.concat([X, X2, X3, X4, df], axis=1)
df = pd.concat([X, X3, df], axis=1)
#df.to_csv('Data/df_selected_features.csv', index=True)

#df=df.drop(columns=['time'])
df=df.drop(columns=['high','low','close','open'])
#df=df.dropna()
#df=df.reset_index(drop=True)


In [ ]:
# Define Split: 100 % // (n_splits + 1): 16,6 % for n_splits=5
tscv = TimeSeriesSplit(n_splits=5) 

sum_fp = 0
sum_tp = 0

for train_index, test_index in tscv.split(df):
    train_data, test_data = df.iloc[train_index], df.iloc[test_index]
    
    # Train data
    x_train = train_data.iloc[:, :-2].values
    y_train = train_data[target].values
    
    # Test data
    x_test = test_data.iloc[:, :-2].values
    y_test = test_data[target].values

    # Scale Data
    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)

    # Hyperparameters
    n_estimators = 80
    class_weight = {0: 5, 1: 6}
    max_features = None
    random_state = 5

    # Initialize RandomForestClassifier
    rf_classifier = RandomForestClassifier(
        n_estimators=n_estimators,
        class_weight=class_weight,
        max_features=max_features,
        random_state=random_state
    )

    # Train Model
    rf_classifier.fit(x_train, y_train)

    # Predict
    y_pred = rf_classifier.predict(x_test)
   
    # Evaluate
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    false_positives = confusion_matrix(y_test, y_pred)[0][1]
    true_positives = confusion_matrix(y_test, y_pred)[1][1]

    sum_fp += false_positives
    sum_tp += true_positives

    precision = precision_score(y_test, y_pred)
    print('WIN/LOSS-Diff:', round(100 * (precision - BreakEvenRatio), 2), '%')
    print('sum_fp:', sum_fp)
    print('sum_tp:', sum_tp)
    print('Ratio total', round(100 * (sum_tp / (sum_fp + sum_tp)), 2))
    print('BreakEvenRatio:', round(BreakEvenRatio, 2))
    print('____________________________________________________________________________________________________________________________')

In [5]:
import pandas as pd

def custom_time_series_split(df, n_splits, test_size=0.1):
    total_samples = len(df)
    test_samples = int(total_samples * test_size)  # Calculate number of samples for the test set
    print("Total Samples:", total_samples)
    print("Test Samples:", test_samples)

    # Calculate split indices
    split_indices = [total_samples - (n_splits - i) * test_samples for i in range(n_splits)]
    print("Split Indices:", split_indices)

    # First split with 50% data
    train_index_1 = df.index[:split_indices[0]]
    test_index_1 = df.index[split_indices[0]:]
    print("Fold 1 Train:", train_index_1, "Test:", test_index_1)
    yield train_index_1, test_index_1

    # Subsequent splits with desired pattern
    for i in range(1, n_splits):
        train_index = df.index[:split_indices[i]]
        if i == n_splits - 1:
            test_index = df.index[split_indices[i]:]  # Using all remaining indices for the last split
        else:
            test_index = df.index[split_indices[i]:split_indices[i + 1]]  # Calculate test index range for subsequent splits
        print("Fold", i+1, "Train:", train_index, "Test:", test_index)

        yield train_index, test_index

# Test the function
df_index = list(range(1, 11))
df = pd.DataFrame(index=df_index)
custom_splits = list(custom_time_series_split(df, n_splits=5))

# Verify the splits
desired_splits = [
    ([1, 2, 3, 4, 5], [6, 7, 8, 9, 10]),
    ([1, 2, 3, 4, 5, 6], [7, 8, 9, 10]),
    ([1, 2, 3, 4, 5, 6, 7], [8, 9, 10]),
    ([1, 2, 3, 4, 5, 6, 7, 8], [9, 10]),
    ([1, 2, 3, 4, 5, 6, 7, 8, 9], [10])
]

matches = all(train.tolist() == desired_splits[i][0] and test.tolist() == desired_splits[i][1] for i, (train, test) in enumerate(custom_splits))
print("Splits match the desired pattern:", matches)


Total Samples: 10
Test Samples: 5
Split Indices: [-15, -10, -5, 0, 5]
Fold 1 Train: Index([], dtype='int64') Test: Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype='int64')
Fold 2 Train: Index([], dtype='int64') Test: Index([1, 2, 3, 4, 5], dtype='int64')
Fold 3 Train: Index([1, 2, 3, 4, 5], dtype='int64') Test: Index([], dtype='int64')
Fold 4 Train: Index([], dtype='int64') Test: Index([1, 2, 3, 4, 5], dtype='int64')
Fold 5 Train: Index([1, 2, 3, 4, 5], dtype='int64') Test: Index([6, 7, 8, 9, 10], dtype='int64')
Splits match the desired pattern: False
